# Импорты, функции из занятия

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from itertools import chain, product

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
def plot_manifold(latent_r, labels=None, alpha=0.5):
    plt.figure(figsize=(10,10))
    if labels is None:
      plt.scatter(latent_r[:, 0], latent_r[:, 1], cmap="tab10", alpha=0.9)
    else:
      plt.scatter(latent_r[:, 0], latent_r[:, 1], c=labels, cmap="tab10", alpha=0.9)
      plt.colorbar()
    plt.show()

def plot_digits(*args, invert_colors=True, digit_size=28, name=None):
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    figure = np.zeros((digit_size * len(args), digit_size * n))

    for i in range(n):
        for j in range(len(args)):
            figure[j * digit_size: (j + 1) * digit_size,
                   i * digit_size: (i + 1) * digit_size] = args[j][i].squeeze()

    if invert_colors:
        figure = 1 - figure

    plt.figure(figsize=(2 * n, 2*len(args)))
    
    plt.imshow(figure, cmap='Greys_r', clim=(0, 1))
    
    plt.grid(False)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if name is not None:
        plt.savefig(name)
    plt.show()

def train(enc, 
          dec,
          loader, 
          optimizer, 
          single_pass_handler, 
          loss_handler,
          epoch, 
          log_interval=500):
    for batch_idx, (data, lab) in enumerate(loader):
        batch_size = data.size(0)
        optimizer.zero_grad()
        data = data.to(device)
        lab = lab.to(device)

        latent, output = single_pass_handler(encoder, decoder, data, lab)

        loss = loss_handler(data, output, latent)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss.item()))
            
def ae_pass_handler(encoder, decoder, data, *args, **kwargs):
    latent = encoder(data)
    recons = decoder(latent)
    return latent, recons

def ae_loss_handler(data, recons, *args, **kwargs):
    return F.binary_cross_entropy(recons, data)

def run_eval(encoder, 
             decoder, 
             loader, 
             single_pass_handler,
             return_real=True, 
             return_recontr=True,
             return_latent=True,
             return_labels=True):
  
    if return_real:
        real = []
    if return_recontr:
        reconstr = []
    if return_latent:
        latent = []
    if return_labels:
        labels = []
    with torch.no_grad():
        for batch_idx, (data, lab) in enumerate(loader):  
            if return_labels:
                labels.append(lab.numpy())
            if return_real:
                real.append(data.numpy())
            
            data = data.to(device)
            lab = lab.to(device)
            rep, rec = single_pass_handler(encoder, decoder, data, lab)
            if return_latent:
                latent.append(rep.to('cpu').numpy())
            if return_recontr:
                reconstr.append(rec.to('cpu').numpy())
    
    result = {}
    if return_real:
        real = np.concatenate(real)
        result['real'] = real.squeeze()
    if return_latent:
        latent = np.concatenate(latent)
        result['latent'] = latent
    if return_recontr:
        reconstr = np.concatenate(reconstr)
        result['reconstr'] = reconstr.squeeze()
    if return_labels:
        labels = np.concatenate(labels)
        result['labels'] = labels
    return result


In [ ]:
#handson-ml2

import matplotlib as mpl
def plot_percent_hist(ax, data, bins):
    counts, _ = np.histogram(data, bins=bins)
    widths = bins[1:] - bins[:-1]
    x = bins[:-1] + widths / 2
    ax.bar(x, counts / len(data), width=widths*0.8)
    ax.xaxis.set_ticks(bins)
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(
        lambda y, position: "{}%".format(int(np.round(100 * y)))))
    ax.grid(True)

  
def plot_activations_histogram(activations, height=1, n_bins=10):
    activation_means = activations.mean(axis=0)
    
    mean = activation_means.mean()
    bins = np.linspace(0, 1, n_bins + 1)

    fig, [ax1, ax2] = plt.subplots(figsize=(10, 3), nrows=1, ncols=2, sharey=True)
    plot_percent_hist(ax1, activations.ravel(), bins)
    ax1.plot([mean, mean], [0, height], "k--", label="Overall Mean = {:.2f}".format(mean))
    ax1.legend(loc="upper center", fontsize=14)
    ax1.set_xlabel("Activation")
    ax1.set_ylabel("% Activations")
    ax1.axis([0, 1, 0, height])
    plot_percent_hist(ax2, activation_means, bins)
    ax2.plot([mean, mean], [0, height], "k--")
    ax2.set_xlabel("Neuron Mean Activation")
    ax2.set_ylabel("% Neurons")
    ax2.axis([0, 1, 0, height])



In [ ]:
class AddGaussianNoise:
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_size):
        super().__init__()
        self.latent_size = latent_size
        hidden_dims = [32, 64, 128, 256, 512]

        # Build Encoder
        modules = []
        in_channels = 1
        for h_dim in hidden_dims[:-1]:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels=in_channels,
                              out_channels=h_dim,
                              kernel_size=3, stride=2 , padding=1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels=256,
                              out_channels=512,
                              kernel_size= 1),
                    nn.BatchNorm2d(512),
                    nn.LeakyReLU())
        )
        modules.append(nn.Flatten())
        modules.append(nn.Linear(hidden_dims[-1] * 4, latent_size))

        self.encoder = nn.Sequential(*modules)      
    
    def forward(self, x):
        x = self.encoder(x)
        return x
        
class Decoder(nn.Module):
    def __init__(self, latent_size):
        super().__init__()

        hidden_dims = [512, 256, 128, 64, 32]
        self.linear = nn.Linear(in_features=latent_size, 
                                out_features=hidden_dims[0])
        
        modules = []
        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride=2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
            )


        modules.append(nn.Sequential(
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels=1,
                                      kernel_size=7, padding=1),
                            nn.Sigmoid()))

        self.decoder = nn.Sequential(*modules)   

        
    def forward(self, x):
        x = self.linear(x)
        x = x.view(-1, 512, 1, 1)
        x = self.decoder(x)
        return x

# Задание 2. Разреженный autoencoder с KL-loss




На занятии мы обсуждали, что разреженный автоэнкодер можно делать двумя путями - при помощи L1 и при помощи KL лосса. На занятии мы сделали с L1  лоссом. 

Ваша задача состоит в том, чтобы реализовать разреженный автоэнкодер с KL-лоссом. 

$$KL(P||Q) =  p(x) \log \dfrac {p(x)} {\hat{p}(x) + \epsilon} + (1 - p(x)) \log \dfrac {(1 - p(x))} {1 - \hat{p}(x) + \epsilon} $$
Обучите автоэнкодер с требованием, чтобы активировалось не более 10% нейронов. 

Учтите, что лосс надо считать по активациям, распределенным от 0 до 1 - то есть надо выполнить преобразование, аналогичное тому, что мы делали ранее



Напоминаем, что в этом случае мы:
 1. Усредняем активации по батчу
 2. Для каждого нейрона таким образом получаем среднюю "вероятность" активироваться
 3. Эта вероятность вряд ли будет в точности равна 0 или 1, но в выражение для KL-loss в знаменатель на всякий случай, добавляем малое число epsilon.
 4. Подсчитанный лосс усредняем по всем нейронам слоя
 5. Сделайте выводы


Постройте графики:

 1. Того, как восстанавливает автоэнкодер полученные изображения
 2. Средних активаций для каждого класса 
 3. Распределения силы активаций в целом и средней силы активации каждого нейрона







В следующих заданиях будет использоваться обычный MNIST

In [ ]:

root = './data'

train_set = dset.MNIST(root=root, 
                       train=True, 
                       transform=torchvision.transforms.ToTensor(),
                       download=True)
test_set = dset.MNIST(root=root, 
                      train=False,
                      transform=torchvision.transforms.ToTensor(),
                      download=True)

test_noise_set = dset.MNIST(root=root, 
                      train=False,
                      transform=torchvision.transforms.Compose([
                          torchvision.transforms.ToTensor(),
                          AddGaussianNoise(0., 0.30)
                      ]),
                      download=True)
train_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=64,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=64,
    shuffle=False)

test_noised_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.Subset(test_noise_set, list(range(64))),
    batch_size=64, 
    shuffle=False)



In [ ]:
def to_01_activation(latent):
  activations = (torch.sigmoid(latent.abs()) - 0.5) * 2
  return activations

def sparse_kl_loss(latent, p=0.10, eps=10e-5):
  activations = to_01_activation(latent)
  phat = activations.mean(axis=0) 
  loss = p * torch.log(p/(phat + eps) ) + (1 - p) * torch.log( (1 - p) / (1 - phat + eps) )
  return loss.mean()



def sparse_ae_pass_handler(encoder, decoder, data, *args, **kwargs):
    latent = encoder(data)
    recons = decoder(latent)
    return latent, recons

def sparse_ae_loss_handler(data, recons, latent, beta=0.1, *args, **kwargs):
    return F.binary_cross_entropy(recons, data) + sparse_kl_loss(latent)

In [ ]:
latent_size = 16 * 16

learning_rate = 1e-4
encoder = Encoder(latent_size=latent_size)
decoder = Decoder(latent_size=latent_size)


encoder = encoder.to(device)
decoder = decoder.to(device)

optimizer = optim.Adam(chain(encoder.parameters(),
                            decoder.parameters()
                           ),
                      lr=learning_rate)

In [ ]:
from functools import partial
for i in range(1, 6):
    train(enc=encoder, 
      dec=decoder, 
      optimizer=optimizer,
      loader=train_loader,
      epoch=i, 
      single_pass_handler=sparse_ae_pass_handler,
      loss_handler=partial(sparse_ae_loss_handler, beta=0.01),
      log_interval=450)

In [ ]:
encoder = encoder.eval() 
decoder = decoder.eval()

In [ ]:
run_res = run_eval(encoder, decoder, test_noised_dataloader, sparse_ae_pass_handler)
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, sparse_ae_pass_handler)
_, axs = plt.subplots(nrows=2, ncols=5, figsize=(16, 5))
activations = to_01_activation(torch.from_numpy(run_res['latent'])).numpy()
for label in range(0, 10):
    
    figure = activations[run_res['labels'] == label].mean(axis=0)
    figure = figure.reshape(16, 16)
    ax = axs[label % 2, label % 5]
    ax.imshow(figure,
            cmap='Greys_r',
            clim=(0, 1))
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
plot_activations_histogram(activations, height=1.)
plt.show()

## Памятка для преподавателя
Студент должен получить в результате разреженный актоенкодер с целевой долей активаций. 
Вывод - KL-loss удобнее при прочих равных